## Demos: Lecture 12

### Demo 1: Shor's algorithm

In [1]:
import pennylane as qml
import numpy as np

from lecture12_helpers import *

/opt/conda/envs/pennylane/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


<img src="fig/shor-flowchart.jpeg" width="300px">

In [3]:
def shors_algorithm(N):
    
    for _ in range(10):
        
        # Choose a candidate a
        a = np.random.choice(list(range(2, N - 1)))
        
        # Check if it's co-prime
        if np.gcd(a, N) != 1:
            p = np.gcd(a, N)
            q = N // p
            return p, q
        
        # If it's co-prime, construct U_Na, run QPE 
        # to get the period
        r = get_period_quantum(N, a)
        
        # If the period is odd, restart
        if r % 2 == 1:
            continue
        
        # If the period is even, compute square root
        x = (a ** (r // 2)) % N
        
        # If non-trivial, find p and q with gcd
        if x != 1 and x != N - 1:
            p = np.gcd(x - 1, N)
            q = np.gcd(x + 1, N)
            return p, q

In [10]:
def get_period_quantum(N, a):
    
    U_Na = get_U_Na(N, a)
    
    n_estimation_wires = 12
    n_target_wires = int(np.log2(len(U_Na)))
    
    estimation_wires = range(n_estimation_wires)
    target_wires = range(n_estimation_wires, n_estimation_wires+n_target_wires)
    
    dev = qml.device('default.qubit', wires=n_estimation_wires+n_target_wires, shots=1)

    @qml.qnode(dev)
    def run_qpe():
        # Initialize state
        qml.PauliX(wires=target_wires[-1])
        
        # Do QPE
        qml.QuantumPhaseEstimation(
            U_Na,
            estimation_wires=estimation_wires,
            target_wires=target_wires
        )
        
        # Return a sample
        return qml.sample(wires=estimation_wires)
    
    candidate_r = []
    
    for _ in range(10):
        output_sample = run_qpe()
        phase = fractional_binary_to_float(output_sample)
        est_r = phase_to_order(phase, N)
        candidate_r.append(est_r)
        
    return max(candidate_r)
        
        
        

In [ ]:
N = 391 # 19, 13

for _ in range(10):
    p, q = shors_algorithm(N)
    if p * q == N:
        print(f"p={p}\nq={q}")
        break

In [11]:
23 * 17

391